<a href="https://colab.research.google.com/github/MUMADE-TADM/S1-python-y-S5-Titanic-/blob/main/rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Yahoo finance](https://finance.yahoo.com/)

In [ ]:
!pip install -U yfinance pandas_datareader

# Datos: valor de cierre del IBEX.

In [ ]:
import pandas as pd
import yfinance as yf
from pandas_datareader import data as pdr

In [ ]:
yf.pdr_override() # <== that's all it takes :-)
data = pdr.get_data_yahoo("^IBEX", start="2020-01-01", end="2020-10-30")

## [Bollinger Bands](https://en.wikipedia.org/wiki/Bollinger_Bands):

El 97% de los valores de un activo se encuentran entre un máximo de $+1.96 \times  \sigma(20\ valores\ anteriores)$ y $-1.96 \times \sigma(20\ valores\  anteriores)$ centrados en la media de los 20 valores anteriores.

In [ ]:
d=pd.DataFrame()
d['R']=data['Close']
d['M']=data['Close'].rolling(20).mean()
d['+M']=d['M']+data['Close'].rolling(20).std()*1.96
d['-M']=d['M']-data['Close'].rolling(20).std()*1.96
d.plot(figsize=(15,5))

## Transformación de los datos para un problema de regresión / clasificación:
- Datos originales $[c_0,\ldots,c_T]$.
- Datos transformados:$[[c_0,\ldots,c_{19}],c_{20}],\ldots,[[c_{T-20},\ldots,c_{T-1}],c_T]$.

In [ ]:
def windowData(s,window_input=1,window_output=1,step=1):
  X=[]
  Y=[]
  #Datos necesarios:
  dn=window_input+window_output
  #Calcular cuantos pasos completos podemos realizar, sin contar el último bloque [input,output].
  ld=int((len(s)-dn)/step)*step
  for i in range(len(s)-ld-1,len(s)-dn,step):
    X.append(s[i:i+window_input])
    Y.append(s[i+window_input:i+dn])
  return X,Y

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

#Escalanos los datos:
dScale = MinMaxScaler()
ldatos=d['R'].values #ldatos.shape=(len(d['R']),).
ldatos=ldatos.reshape((len(d['R']),1)) #ldatos.shape=(len(d['R']),1).
dScale.fit(ldatos)
ldatos=dScale.transform(ldatos).reshape((len(ldatos),))

#Contruimos los conjunto de datos X e Y:
X,Y=windowData(ldatos,window_input=20,window_output=1,step=1)
X=np.array(X)
Y=np.array(Y)

#Patimos los conjuntos en entranamiento y test:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.33,shuffle=False, random_state=42)

In [ ]:
X.shape

## Ajustamos los datos!!
Los datos en las **redes recurrentes** son tensores de 3 dimensiones
\[número de ejemplos por batch, tamaño del paso (tamaño del dataset), número de características \].

**\[size,step,features\] !!**

Consideramos un **step** de 20 con una única **característica** (cant. de parámetros que queremos tener).


In [ ]:
XRNN=X.reshape((X.shape[0],X.shape[1],1))
XRNN.shape

## Librerías necesarias:
- [Keras](https://keras.io/).
- [Tensorflow](https://www.tensorflow.org/).

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input,SimpleRNN
from tensorflow.keras import metrics

## Definimos la Red:

Definimos nuestro perceptron con las siguientes capas (**layers**):
- Modelo secuencial [Sequential](https://keras.io/api/models/sequential/).
- Capa de entrada: [SimpleRNN](https://keras.io/api/layers/recurrent_layers/simple_rnn/).
- Capa Densa: [Dense](https://keras.io/api/layers/core_layers/dense/).

La capa **SimpleRNN** define una unidad recurente con **unit** neuronas de salida y una entrada de **input_shape**.

**Recurrente básica :**

In [ ]:
#Definimos nuestro modelo y vamos añadiendo capas. 
rnn = Sequential() 
rnn.add(SimpleRNN(units=32, activation="relu",input_shape=(20,1))) #units=32 -> significa que la salida de esta capa va a ser 32. 
rnn.add(Dense(1))
rnn.compile(loss='mean_squared_error', optimizer='rmsprop')
rnn.summary()

Las *va* empaquetadas de las diapos tienen un valor de  1088.

## Compilamos la red:
Definido el **perceptrón** lo tendremos que compilar indicando:
- Que optimizador utilizar.
- Que función de error (pérdida o **loss**).
- Que métricas observar cuando lo entrenemos.

Función [**compile**](https://keras.io/api/models/model_training_apis/#compile-method).

In [ ]:
h=rnn.fit(XRNN,Y,validation_split=0.30, epochs=100,batch_size=5)
#En cada época nos dice el error que va cometiendo. 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(h.history['loss'])
plt.plot(h.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
#Vemos que funciona bien el test. El train empieza mal pero luego bien. 

## Utilizar la red:
Utilizar el perceptron aprendido con el método [**predict**](https://keras.io/api/models/model_training_apis/#predict-method).

In [ ]:
Y0=rnn.predict(XRNN)

In [ ]:
r=pd.DataFrame(Y)
r['RNN']=pd.DataFrame(Y0)
r.plot(figsize=(15,5))

La diferencia conceptual entre percepton y este, es que el perceptron no tiene memoria, no tiene datos del pasado, sólo tiene lo que le hemos metido. Mientras que las redes neuronales recurrentes sí tienen concepto de pasado. De hecho, se lo pasan a la siguiente etapa. 

La red neronal recurrente utiliza lo mismo que perceptron pero el aquí el backd. tiene problemas. Debido a esto, se tiene que empezar a hacer cosas como la normalización para solventar esto. Como por ej: Batch - normalización -> normaliza todos los datos a trabajar a partir de ese bacht. Este trabaja con 0 y 1).

## Salvar y Cargar Redes:
- Salvar mediante el método [**save**](https://keras.io/api/models/model_saving_apis/#save-method).
- Recuperar un modelo con el método [**load_model**](keras.models.load_model).

(Guardamos la red).

In [ ]:
rnn.save('rnn.h5')
rnn2=keras.models.load_model('rnn.h5')